In [11]:
%matplotlib inline
from __future__ import division
from matplotlib import pyplot as plt
import numpy as np
import cv2
fn = 'theFieldHouse.jpg'
img = cv2.imread(fn)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
H, W = img.shape[:2]
H, W

(375, 500)

In [12]:
cv2.CV_PI = 3.1415926535897932384626433832795
thrs1 = 50
thrs2 = 200
cannyEdge = cv2.Canny(gray, thrs1, thrs2, apertureSize=3)
cv2.imshow('Canny', cannyEdge)
lines = cv2.HoughLinesP(cannyEdge,1, cv2.CV_PI/180, 80, 30, 10)

In [13]:
color_dst = cv2.cvtColor(cannyEdge,cv2.COLOR_GRAY2BGR)
for i in range(0,lines.shape[0]):
    cv2.line(color_dst,(lines[i][0][0],lines[i][0][1]), (lines[i][0][2],lines[i][0][3]),(0,0,255),3,8 )
cv2.imshow('withVert', color_dst)
lines.shape

(103, 1, 4)

In [14]:
vertLines = []
for i in range(0,lines.shape[0]):
    if lines[i][0][0] == lines[i][0][2]:
        vertLines.extend([i])
lines = np.delete(lines, vertLines, axis=0)

In [15]:
color_dst = cv2.cvtColor(cannyEdge,cv2.COLOR_GRAY2BGR)
for i in range(0,lines.shape[0]):
    cv2.line(color_dst,(lines[i][0][0],lines[i][0][1]), (lines[i][0][2],lines[i][0][3]),(0,0,255),3,8 )
lines.shape

(79, 1, 4)

In [16]:
cv2.imshow('withoutVert', color_dst)
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [ ]:
import sys
from PIL import Image, ImageDraw
tst = gray.copy()
tst[:,:] = 0
tst = Image.fromarray(tst)
draw = ImageDraw.Draw(tst)




g=200
d=400
a = g - d*g/(d + W)
b = d*(H - g)/(d + W) + g
print a,b
# intersection points
ip = np.arange(50,H,30)
for h in ip:
    print h
    if 0 <= h < a:
        x = d*h/(g-h)
        draw.line((0, h, x, 0), fill=255)
    elif a <= h < b:
        y = (d*g - (g - h)*(d + W))/d
        draw.line((0, h, W, y), fill=255)
    elif b <= h:
        pass
#         x = (d*(H-g) - d*(b-g))/(b-g)
#         draw.line((0, h, x, H), fill=128)
del draw
tst.save('tst.png', "PNG")

In [ ]:
#tstImg = np.reshape(list(tst.getdata()),[H,W])
gray = cv2.imread('tst.png')
gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
cv2.imshow('gray', gray)
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [ ]:
ksize = 3
# dx = cv2.Sobel(gray, cv2.CV_16S, 1, 0, ksize)
# dy = cv2.Sobel(gray, cv2.CV_16S, 0, 1, ksize)
dx = cv2.Scharr(gray, cv2.CV_16S, 1, 0)
dy = cv2.Scharr(gray, cv2.CV_16S, 0, 1)
with np.errstate(divide='ignore', invalid='ignore'):
    m = np.true_divide(dy,dx)
m[ dx == 0 ] = 0
# # http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_features_harris/py_features_harris.html
# k = 0.04
# eig = cv2.cornerEigenValsAndVecs(gray, 5, 3)
# eigVal0 = eig[:,:,0]
# eigVal1 = eig[:,:,1]
# R = np.multiply(eigVal0,eigVal1) - k*(eigVal0 + eigVal1)**2
# isEdge = R < 0

# # discard non-edges
# m[np.logical_not(isEdge)] = 0
# cv2.imshow('posSlope', np.multiply(m, m > 0))
# cv2.imshow('negSlope', -1*np.multiply(m, m < 0))
# cv2.waitKey(0) 
# cv2.destroyAllWindows()

hwIdx = np.unravel_index(range(0,H*W),(H, W))
hIdx = hwIdx[0]
wIdx = hwIdx[1]
m = np.reshape(m,[-1,])


#isEdge = np.reshape(isEdge,[-1,])


# # bl2tr = -
# # limit slope values
# m_thresh = 0.5*h/w
# selectEdge = np.logical_and(isEdge, np.absolute(m) < m_thresh)
# # only consider selected edges
# hIdx = hIdx[selectEdge]
# wIdx = wIdx[selectEdge]
# m = m[selectEdge]
# a = np.column_stack( ( np.ones((m.shape[0],)), m ) )
# b = hIdx - m*wIdx
# x = np.linalg.lstsq(a,b)
# x

# bl2tr = +
# limit slope values
m_thresh = 0.8*H/W
#cannyEdge =np.reshape(cannyEdge,[-1,])
#selectEdge = np.logical_and(np.logical_and(isEdge, np.absolute(m) < m_thresh), cannyEdge > 0)

selectEdge = m != 0
#selectEdge = isEdge

# only consider selected edges
hIdx = hIdx[selectEdge]
wIdx = wIdx[selectEdge]
m = m[selectEdge]
# Solve Ax = B
A = np.column_stack( ( np.ones((m.shape[0],)), -m ) )
B = hIdx + m*wIdx
x = np.linalg.lstsq(A, B)
g = x[0][0]
# Why x[0][1] is negative?!?
# And why is Sobel output negated?? ... related?
d = x[0][1]
a = g - d*g/(d + W)
b = d*(H - g)/(d + W) + g
a,b,g,d

In [ ]:
A.shape,B.shape

In [ ]:
Z=np.column_stack((A,B))

In [ ]:
A

In [ ]:
USV=np.linalg.svd(Z)

In [ ]:
USV

In [ ]:
U=USV[0]
S=USV[1]
V=USV[2]

In [ ]:
VXY=V[0:2,2]
VXY

In [ ]:
VYY=V[2,2]
VYY

In [ ]:
B = -VXY/VYY
B

In [ ]:
plt.plot(m)
plt.show()

In [ ]:
m

In [ ]:
import math
import matplotlib.pyplot as plt

from skimage.io import imread, imsave
from skimage import transform as tf

src = np.array([[0, 0], [0, H], [W, H], [W, 0]])
dst = np.array([[0, a], [0, b], [W, H], [W, 0]])

tform3 = tf.ProjectiveTransform()
tform3.estimate(src, dst)
warped = tf.warp(gray, tform3, output_shape=gray.shape)
#cv2.imshow('warped',warped)

fig, ax = plt.subplots(nrows=2, figsize=(8, 3))

ax[0].imshow(gray, cmap=plt.cm.gray)
ax[0].plot(dst[:, 0], dst[:, 1], '.r')
ax[1].imshow(warped, cmap=plt.cm.gray)

for a in ax:
    a.axis('off')
    
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [ ]:
x[0][0], x[0][1]

In [ ]:
edgeSlopes = np.multiply(isEdge,m)

In [ ]:
posSlopeEdge = np.multiply(edgeSlopes, edgeSlopes > 0)

In [ ]:
thrs1 = 100
thrs2 = 300
cannyEdge = cv2.Canny(gray, thrs1, thrs2, apertureSize=3)
cv2.imshow('Canny', edge)
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [ ]:
negSlopeEdge = np.multiply(edgeSlopes, edgeSlopes < 0)

In [ ]:
min(negSlopeEdge)

In [ ]:
aa = np.array([[1,2],[3,4]])
np.reshape(aa,[-1,])

In [ ]:
xy=np.unravel_index(range(0,2*2),(2,2))
xy

In [ ]:
aa[xy]

In [ ]:
aa[0,0],aa[0,1],aa[1,0],aa[1,1]

In [ ]:
a.shape,max(wIdx),max(hIdx)

In [ ]:
edgeSlopes = np.multiply(isEdge,m)
posSlopeEdge = np.multiply(edgeSlopes, edgeSlopes > 0)
negSlopeEdge = np.multiply(edgeSlopes, edgeSlopes < 0)
cv2.imshow('posSlope', posSlopeEdge)
cv2.imshow('negSlope', -1*negSlopeEdge)
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [ ]:
m[ dx == 0 ] = 0

In [ ]:
np.sum(1*(posSlopeEdge!=0))

In [ ]:
a = np.column_stack( ( np.ones((m.shape[0],)), -m ) )
b = hIdx - m*wIdx
c = np.linalg.lstsq(a,b)
c

In [ ]:
k = 0.04
eig = cv2.cornerEigenValsAndVecs(gray, 5, 3)
eigVal0 = eig[:,:,0]
eigVal1 = eig[:,:,1]
x0 = eig[:,:,2]
y0 = eig[:,:,3]
x1 = eig[:,:,4]
y1 = eig[:,:,5]
# http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_features_harris/py_features_harris.html
R = np.multiply(eigVal0,eigVal1) - k*(eigVal0 + eigVal1)**2
isEdge = R < 0
with np.errstate(divide='ignore', invalid='ignore'):
    m0 = np.true_divide(y0,x0)
m0[ x0 == 0 ] = 0
with np.errstate(divide='ignore', invalid='ignore'):
    m1 = np.true_divide(y1,x1)
m1[ x1 == 0 ] = 0
m = np.multiply(eigVal0 > eigVal1, m0) + np.multiply(eigVal1 > eigVal0, m1)
hwIdx = np.unravel_index(range(0,h*w),(h,w))
hIdx = hwIdx[0]
wIdx = hwIdx[1]
m = np.reshape(m,[-1,])
isEdge = np.reshape(isEdge,[-1,])
# only consider edges
hIdx = hIdx[isEdge]
wIdx = wIdx[isEdge]
m = m[isEdge]


# edgeSlopes = np.multiply(isEdge,m)
# posSlopeEdge = np.multiply(edgeSlopes, edgeSlopes > 0)
# negSlopeEdge = np.multiply(edgeSlopes, edgeSlopes < 0)
# cv2.imshow('posSlope', posSlopeEdge)
# cv2.imshow('negSlope', -1*negSlopeEdge)
# cv2.waitKey(0) 
# cv2.destroyAllWindows()



In [ ]:
a = np.column_stack( ( np.ones((m.shape[0],)), -m ) )
b = hIdx - m*wIdx
c = np.linalg.lstsq(a,b)
c

In [ ]:
-m

In [ ]:

b.shape

In [ ]:
c = np.linalg.lstsq(a,b)
c

In [ ]:
cv2.imshow('edges', edges)
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [ ]:

eigen = eigen.reshape(h, w, 3, 2)  # [[e1, e2], v1, v2]
flow = eigen[:,:,2]

vis = img.copy()
vis[:] = (192 + np.uint32(vis)) / 2
d = 12
points =  np.dstack( np.mgrid[d/2:w:d, d/2:h:d] ).reshape(-1, 2)
for x, y in points:
   vx, vy = np.int32(flow[y, x]*d)
   cv2.line(vis, (x-vx, y-vy), (x+vx, y+vy), (0, 0, 0), 1)
cv2.imshow('input', img)
cv2.imshow('flow', vis)
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [ ]:
a=np.array([[1,2],[3,4]])
a

In [ ]:
np.linalg.matrix_power(a,2)

In [ ]:
a**2